In [5]:
import pandas as pd
import numpy as np
import talib
from kiteconnect import KiteConnect
from apscheduler.schedulers.blocking import BlockingScheduler
import telegram
from twilio.rest import Client
import openpyxl
import requests
import json
import datetime
import yfinance as yf
from sklearn.ensemble import RandomForestClassifier

print("\U0001F680 Trading bot started... Press Ctrl+C to stop.")

# Initialize Zerodha API
kite = KiteConnect(api_key="your_api_key")
kite.set_access_token("your_access_token")

# Telegram Alert Setup
TELEGRAM_BOT_TOKEN = "your_telegram_bot_token"
TELEGRAM_CHAT_ID = "your_telegram_chat_id"

def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    data = {"chat_id": TELEGRAM_CHAT_ID, "text": message}
    requests.post(url, data=data)

# WhatsApp Alert Setup
TWILIO_SID = "your_twilio_sid"
TWILIO_AUTH_TOKEN = "your_twilio_auth_token"
TWILIO_WHATSAPP_NUMBER = "whatsapp:+your_twilio_number"
YOUR_WHATSAPP_NUMBER = "whatsapp:+your_number"

def send_whatsapp_message(message):
    client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)
    client.messages.create(
        body=message,
        from_=TWILIO_WHATSAPP_NUMBER,
        to=YOUR_WHATSAPP_NUMBER
    )

# Fetch NSE Stocks Automatically
def get_nse_stocks():
    nse_url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(nse_url, headers=headers)
    data = response.json()
    return [stock['symbol'] for stock in data['data']]

# Candlestick Pattern Detection
def detect_patterns(df):
    patterns = {
        "Double Top": talib.CDLDTOP,
        "Double Bottom": talib.CDLDBL,
        "Cup & Handle": talib.CDLCUPWITHHANDLE,
        "Bullish Engulfing": talib.CDLENGULFING,
        "Bearish Engulfing": talib.CDLENGULFING,
    }
    signals = {}
    for pattern, func in patterns.items():
        signals[pattern] = func(df['Open'], df['High'], df['Low'], df['Close'])
    return signals

# AI-based Stock Selection
def ai_stock_selection():
    stock_list = get_nse_stocks()
    features = []
    labels = []
    for stock in stock_list:
        data = yf.download(stock, period="1y", interval="1d")
        data['Returns'] = data['Close'].pct_change()
        features.append(data[['Open', 'High', 'Low', 'Close', 'Volume']].values)
        labels.append(np.sign(data['Returns'].values))
    model = RandomForestClassifier()
    model.fit(np.vstack(features), np.hstack(labels))
    return stock_list[np.argmax(model.predict(features))]

# Trailing Stop Loss
def calculate_trailing_stop(current_price, entry_price):
    return entry_price + ((current_price - entry_price) / 2)

# Multi-Timeframe Analysis
def check_multi_timeframe(symbol):
    timeframes = ['1h', '4h', '1d']
    signals = []
    for tf in timeframes:
        df = yf.download(symbol, period='7d', interval=tf)
        signals.append(detect_patterns(df))
    return all(signals)

# Trade Execution
def execute_trade(symbol):
    entry_price = kite.ltp(f'NSE:{symbol}')['NSE:{symbol}']['last_price']
    stop_loss = entry_price * 0.98
    take_profit = entry_price * 1.05

    order = kite.place_order(
        variety=kite.VARIETY_REGULAR,
        exchange=kite.EXCHANGE_NSE,
        tradingsymbol=symbol,
        transaction_type=kite.TRANSACTION_TYPE_BUY,
        quantity=1,
        order_type=kite.ORDER_TYPE_LIMIT,
        price=entry_price,
        product=kite.PRODUCT_MIS,
        validity=kite.VALIDITY_DAY
    )
    
    message = (f"Stock Name: {symbol}\n"
               f"Entry Point: {entry_price}\n"
               f"Stop Loss: {stop_loss}\n"
               f"Take Profit Levels: TP1 {take_profit*1.02}, TP2 {take_profit*1.04}, TP3 {take_profit*1.06}\n")
    send_telegram_message(message)
    send_whatsapp_message(message)

# Auto Hedge for Risk Management
def auto_hedge():
    vix = yf.download('^VIX', period='7d', interval='1d')['Close'].iloc[-1]
    if vix > 25:
        execute_trade('HDFCBANK')

# News Sentiment Analysis
def fetch_sentiment(symbol):
    response = requests.get(f'https://newsapi.org/v2/everything?q={symbol}&apiKey=your_newsapi_key')
    data = response.json()
    return 'positive' if any('rise' in article['title'].lower() for article in data['articles']) else 'negative'

# Auto-Daily Trade Report
def generate_trade_report():
    df = pd.read_csv('trade_history.csv')
    df.to_excel('daily_trade_report.xlsx')
    send_telegram_message("Daily trade report generated.")

# Scheduler to Run Bot
scheduler = BlockingScheduler()
scheduler.add_job(lambda: execute_trade(ai_stock_selection()), 'interval', minutes=5)
scheduler.add_job(generate_trade_report, 'cron', hour=23, minute=59)
scheduler.start()


🚀 Trading bot started... Press Ctrl+C to stop.


KeyboardInterrupt: 